In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/rajatmaliwal2913/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('/Users/rajatmaliwal2913/Downloads/Perovskite_Stability_with_features.csv')
data.head()

Material Composition A site #1 A site #2 A site #3 B site #1 B site #2  \
0          Ba1Sr7V8O24        Ba        Sr       NaN         V       NaN   
1      Ba2Bi2Pr4Co8O24        Ba        Bi        Pr        Co       NaN   
2         Ba2Ca6Fe8O24        Ba        Ca       NaN        Fe       NaN   
3      Ba2Cd2Pr4Ni8O24        Ba        Cd        Pr        Ni       NaN   
4         Ba2Dy6Fe8O24        Ba        Dy       NaN        Fe       NaN   

  B site #3 X site  Number of elements  energy_above_hull (meV/atom)  ...  \
0       NaN      O                   4                     29.747707  ...   
1       NaN      O                   5                    106.702335  ...   
2       NaN      O                   4                    171.608093  ...   
3       NaN      O                   5                    284.898190  ...   
4       NaN      O                   4                    270.007913  ...   

   host_Asite0_IsBCC  host_Asite0_IsCubic  host_Asite0_IsAlkali  \
0                  0                    0                     0   
1                  0                    0                     0   
2                  0                    0                     0   
3                  0                    0                     0   
4                  0                    0                     0   

   host_Asite0_OrbitalD  host_Asite0_NsValence  host_Bsite0_At. #  \
0                     0                      2                 23   
1                     0                      2                 27   
2                     0                      2                 26   
3                     0                      2                 28   
4                     0                      2                 26   

   host_Bsite0_IsHexagonal  host_Bsite0_IsNoblegas  Asite_IsAlkali_max  \
0                        0                       0                   1   
1                        1                       0                   1   
2                        0                       0                   1   
3                        0                       0                   1   
4                        0                       0                   1   

   Bsite_IsMetal_max  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  

[5 rows x 81 columns]

In [3]:
data.columns = [c.replace(' ', '_') for c in data.columns]
data.columns = [c.replace('#', '') for c in data.columns]
data.columns = [c.replace('.', '') for c in data.columns]

In [4]:
data = data.drop(['X_site','B_site_3', 'A_site_3','A_site_2', 'B_site_2', 'Asite_IsBoron_weighted_avg','Asite_IsHalogen_weighted_avg', 'host_Asite0_IsCubic','Asite_IsPnictide_weighted_avg' ,'host_Asite0_IsAlkali', 'host_Asite0_NsValence', 'host_Bsite0_IsNoblegas', 'Bsite_IsMetal_max', 'Asite_GSmagmom_min', 'Material_Composition', 'ICSDVolume_AB_avg', 'First_Ionization_Potential_(V)_AB_avg', 'Asite_Atomic_Volume_(cm³/mol)_max','Asite_IsAlkali_max','host_Bsite0_At_'], axis = 1)

In [5]:
y = data.iloc[:, data.columns == 'energy_above_hull_(meV/atom)'].values
X = data.drop("energy_above_hull_(meV/atom)", axis = 1)
print(X.shape)
print(y)

(1929, 60)
[[ 29.74770732]
 [106.702335  ]
 [171.6080933 ]
 ...
 [  0.        ]
 [  0.        ]
 [  0.        ]]


In [6]:
cols = ['A_site_1', 'B_site_1']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 1)


In [7]:
from sklearn.model_selection import train_test_split

test_size = 0.2

X1, X_test, y1, y_test = train_test_split(X, y, test_size=test_size, random_state=42)


print("X_train shape:", X1.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y1.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1543, 60)
X_test shape: (386, 60)
y_train shape: (1543, 1)
y_test shape: (386, 1)


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X1)
X_test_scaled = scaler.transform(X_test)
X_scaled_df = pd.DataFrame(X_scaled, columns=X1.columns)

In [41]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                            colsample_bytree = 0.3, 
                            learning_rate = 0.1,
                            max_depth = 4, 
                            alpha = 10, 
                            n_estimators = 1000)

xgb_reg.fit(X_scaled_df, y1)

y_pred = xgb_reg.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Coefficient of Determination (R^2):", r2)
print(np.sqrt(mse))

Mean Squared Error (MSE): 1440.6497500769485
Coefficient of Determination (R^2): 0.865394527319749
37.95589216547213
